# Deployment (graph) level Metadata


## Prerequisites

 * A kubernetes cluster with kubectl configured
 * curl
 * pygmentize
 

## Setup Seldon Core

Use the setup notebook to [Setup Cluster](seldon_core_setup.ipynb) to setup Seldon Core with an ingress.

In [1]:
!kubectl create namespace seldon

Error from server (AlreadyExists): namespaces "seldon" already exists


In [2]:
!kubectl config set-context $(kubectl config current-context) --namespace=seldon

Context "kind-seldon" modified.


## Used model

In this example notebook we will use a dummy node that can serve multiple purposes in the graph.

The model will read its metadata from environmental variable (this is done automatically).

Actual logic that happens on each of this endpoint is not subject of this notebook.

We will only concentrate on graph-level metadata that orchestrator constructs from metadata reported by each node.

In [3]:
!pygmentize models/generic-node/Node.py

import logging
import random
import os


NUMBER_OF_ROUTES = int(os.environ.get("NUMBER_OF_ROUTES", "2"))


class Node:
    def predict(self, features, names=[], meta=[]):
        logging.info(f"model features: {features}")
        logging.info(f"model names: {names}")
        logging.info(f"model meta: {meta}")
        return features.tolist()

    def input_transformer(self, features, names=[], meta=[]):
        return self.predict(features, names, meta)

    def output_transformer(self, features, names=[], meta=[]):
        return self.predict(features, names, meta)

    def aggregate(self, features, names=[], meta=[]):
        logging.info(f"model features: {features}")
        logging.info(f"model names: {names}")
        logging.info(f"model meta: {meta}")
        return [x.tolist() for x in features]

    def route(self, features, names=[], meta=[]):
        logging.info(f"model features: {features}")
        logging.info(f"model names: {names}")
        logging.info(f"model meta

## Single Model

In case of single-node graph model-level `inputs` and `outputs`, `x` and `y`, will simply be also the deployment-level `graphinputs` and `graphoutputs`.

![single](./images/single.png)

In [5]:
%%writefile graph-metadata/single.yaml

apiVersion: machinelearning.seldon.io/v1
kind: SeldonDeployment
metadata:
  name: graph-metadata-single
spec:
  name: test-deployment
  predictors:
  - componentSpecs:
    - spec:
        containers:
        - image: seldonio/metadata-generic-node:0.2
          name: model
          env:
          - name: MODEL_METADATA
            value: |
              ---
              name: single-node
              versions: [ generic-node/v0.2 ]
              platform: seldon-custom
              inputs:
              - datatype: BYTES
                name: input
                shape: [ 1, 10 ]
              outputs:
              - datatype: BYTES
                name: output
                shape: [ 1 ]
    graph:
      name: model
      type: MODEL
      children: []        
    name: example
    replicas: 1

Overwriting graph-metadata/single.yaml


In [6]:
%%bash

kubectl apply -f graph-metadata/single.yaml

dep_name=$(kubectl get deploy -l seldon-deployment-id=graph-metadata-single -o jsonpath='{.items[0].metadata.name}')
kubectl rollout status deploy/${dep_name}

seldondeployment.machinelearning.seldon.io/graph-metadata-single configured
deployment "graph-metadata-single-example-0-model" successfully rolled out


### Graph Level

Graph level metadata is available at the `api/v1.0/metadata` endpoint of your deployment:


In [7]:
import requests
meta = requests.get("http://localhost:8003/seldon/seldon/graph-metadata-single/api/v1.0/metadata")
meta = meta.json()

assert meta == {
    'name': 'example',
     'models': {'model': {'name': 'single-node',
       'platform': 'seldon-custom',
       'versions': ['generic-node/v0.2'],
       'inputs': [{'datatype': 'BYTES', 'name': 'input', 'shape': [1, 10]}],
       'outputs': [{'datatype': 'BYTES', 'name': 'output', 'shape': [1]}]}},
     'graphinputs': [{'datatype': 'BYTES', 'name': 'input', 'shape': [1, 10]}],
     'graphoutputs': [{'datatype': 'BYTES', 'name': 'output', 'shape': [1]}]
}

meta

{'name': 'example',
 'models': {'model': {'name': 'single-node',
   'platform': 'seldon-custom',
   'versions': ['generic-node/v0.2'],
   'inputs': [{'datatype': 'BYTES', 'name': 'input', 'shape': [1, 10]}],
   'outputs': [{'datatype': 'BYTES', 'name': 'output', 'shape': [1]}]}},
 'graphinputs': [{'datatype': 'BYTES', 'name': 'input', 'shape': [1, 10]}],
 'graphoutputs': [{'datatype': 'BYTES', 'name': 'output', 'shape': [1]}]}

### Model Level

Compare with `model` metadata available at the `api/v1.0/metadata/model`:

In [8]:
import requests
meta = requests.get("http://localhost:8003/seldon/seldon/graph-metadata-single/api/v1.0/metadata/model")
meta = meta.json()

assert meta == {
    'inputs': [{'datatype': 'BYTES', 'name': 'input', 'shape': [1, 10]}],
    'name': 'single-node',
    'outputs': [{'datatype': 'BYTES', 'name': 'output', 'shape': [1]}],
    'platform': 'seldon-custom',
    'versions': ['generic-node/v0.2'],
    'apiVersion': 'v2',
    }

meta

{'apiVersion': 'v2',
 'inputs': [{'datatype': 'BYTES', 'name': 'input', 'shape': [1, 10]}],
 'name': 'single-node',
 'outputs': [{'datatype': 'BYTES', 'name': 'output', 'shape': [1]}],
 'platform': 'seldon-custom',
 'versions': ['generic-node/v0.2']}

## Two-Level Graph

In two-level graph graph output of the first model is input of the second model, `x2=y1`.

The graph-level input `x` will be first model’s input `x1` and graph-level output `y` will be the last model’s output `y2`.



![two-level](./images/two-level.png)

In [9]:
%%writefile graph-metadata/two-levels.yaml

apiVersion: machinelearning.seldon.io/v1
kind: SeldonDeployment
metadata:
  name: graph-metadata-two-levels
spec:
  name: test-deployment
  predictors:
  - componentSpecs:
    - spec:
        containers:
        - image: seldonio/metadata-generic-node:0.2
          name: node-one
          env:
          - name: MODEL_METADATA
            value: |
              {"name": "node-one",
               "versions": ["generic-node/v0.2"],
               "platform": "seldon-custom",
               "inputs": [{"name": "input", "datatype": "BYTES", "shape": [1, 10]}],
               "outputs": [{"name": "output", "datatype": "BYTES", "shape": [1, 20]}]}            
        - image: seldonio/metadata-generic-node:0.2
          name: node-two
          env:
          - name: MODEL_METADATA
            value: |
              {"name": "node-two",
               "versions": ["generic-node/v0.2"],
               "platform": "seldon-custom",
               "inputs": [{"name": "input", "datatype": "BYTES", "shape": [1, 20]}],
               "outputs": [{"name": "output", "datatype": "BYTES", "shape": [1]}]}               
    graph:
      name: node-one
      type: MODEL
      children:
      - name: node-two
        type: MODEL
        children: []   
    name: example
    replicas: 1

Overwriting graph-metadata/two-levels.yaml


In [10]:
%%bash

kubectl apply -f graph-metadata/two-levels.yaml

dep_name=$(kubectl get deploy -l seldon-deployment-id=graph-metadata-two-levels -o jsonpath='{.items[0].metadata.name}')
kubectl rollout status deploy/${dep_name}

seldondeployment.machinelearning.seldon.io/graph-metadata-two-levels configured
deployment "graph-metadata-two-levels-example-0-node-one-node-two" successfully rolled out


In [11]:
import requests
meta = requests.get("http://localhost:8003/seldon/seldon/graph-metadata-two-levels/api/v1.0/metadata")
meta = meta.json()

assert meta == {
    'name': 'example',
    'models': {
        'node-one': {
            'name': 'node-one',
            'platform': 'seldon-custom',
            'versions': ['generic-node/v0.2'],
            'inputs': [{'datatype': 'BYTES', 'name': 'input', 'shape': [1, 10]}],
            'outputs': [{'datatype': 'BYTES', 'name': 'output', 'shape': [1, 20]}]
        },
        'node-two': {
            'name': 'node-two',
            'platform': 'seldon-custom',
            'versions': ['generic-node/v0.2'],
            'inputs': [{'datatype': 'BYTES', 'name': 'input', 'shape': [1, 20]}],
            'outputs': [{'datatype': 'BYTES', 'name': 'output', 'shape': [1]}]
        }
    },
    'graphinputs': [{'datatype': 'BYTES', 'name': 'input', 'shape': [1, 10]}],
    'graphoutputs': [{'datatype': 'BYTES', 'name': 'output', 'shape': [1]}]}

meta

{'name': 'example',
 'models': {'node-one': {'name': 'node-one',
   'platform': 'seldon-custom',
   'versions': ['generic-node/v0.2'],
   'inputs': [{'datatype': 'BYTES', 'name': 'input', 'shape': [1, 10]}],
   'outputs': [{'datatype': 'BYTES', 'name': 'output', 'shape': [1, 20]}]},
  'node-two': {'name': 'node-two',
   'platform': 'seldon-custom',
   'versions': ['generic-node/v0.2'],
   'inputs': [{'datatype': 'BYTES', 'name': 'input', 'shape': [1, 20]}],
   'outputs': [{'datatype': 'BYTES', 'name': 'output', 'shape': [1]}]}},
 'graphinputs': [{'datatype': 'BYTES', 'name': 'input', 'shape': [1, 10]}],
 'graphoutputs': [{'datatype': 'BYTES', 'name': 'output', 'shape': [1]}]}

## Combiner of two models

In graph with the `combiner` request is first passed to combiner's children and before it gets aggregated by the `combiner` itself.

Input `x` is first passed to both models and their outputs `y1` and `y2` are passed to the combiner. 

Combiner's output `y` is the final output of the graph.

![combiner](./images/combiner.png)

In [12]:
%%writefile graph-metadata/combiner.yaml

apiVersion: machinelearning.seldon.io/v1
kind: SeldonDeployment
metadata:
  name: graph-metadata-combiner
spec:
  name: test-deployment
  predictors:
  - componentSpecs:
    - spec:
        containers:
        - image: seldonio/metadata-generic-node:0.2
          name: node-combiner
          env:
          - name: MODEL_METADATA
            value: |
              {"name": "node-combiner",
               "versions": ["generic-node/v0.2"],
               "platform": "seldon-custom",
               "inputs": [
                   {"name": "input-1", "datatype": "BYTES", "shape": [1, 20]},
                   {"name": "input-2", "datatype": "BYTES", "shape": [1, 30]}
               ],
               "outputs": [{"name": "output", "datatype": "BYTES", "shape": [1]}]}
        - image: seldonio/metadata-generic-node:0.2
          name: node-one
          env:
          - name: MODEL_METADATA
            value: |
              {"name": "node-one",
               "versions": ["generic-node/v0.2"],
               "platform": "seldon-custom",
               "inputs": [{"name": "input", "datatype": "BYTES", "shape": [1, 10]}],
               "outputs": [{"name": "output", "datatype": "BYTES", "shape": [1, 20]}]}            
        - image: seldonio/metadata-generic-node:0.2
          name: node-two
          env:
          - name: MODEL_METADATA
            value: |
              {"name": "node-two",
               "versions": ["generic-node/v0.2"],
               "platform": "seldon-custom",
               "inputs": [{"name": "input", "datatype": "BYTES", "shape": [1, 10]}],
               "outputs": [{"name": "output", "datatype": "BYTES", "shape": [1, 30]}]}               
    graph:
      name: node-combiner
      type: COMBINER
      children:
      - name: node-one
        type: MODEL
        children: []   
      - name: node-two
        type: MODEL
        children: []   
    name: example
    replicas: 1

Overwriting graph-metadata/combiner.yaml


In [13]:
%%bash

kubectl apply -f graph-metadata/combiner.yaml

dep_name=$(kubectl get deploy -l seldon-deployment-id=graph-metadata-combiner -o jsonpath='{.items[0].metadata.name}')
kubectl rollout status deploy/${dep_name}

seldondeployment.machinelearning.seldon.io/graph-metadata-combiner configured
deployment "seldon-f3fb28d397222d10648e6c1c7e0bea9e" successfully rolled out


In [14]:
import requests
meta = requests.get("http://localhost:8003/seldon/seldon/graph-metadata-combiner/api/v1.0/metadata")
meta = meta.json()

assert meta == {
    'name': 'example',
    'models': {
        'node-combiner': {
            'name': 'node-combiner',
            'platform': 'seldon-custom',
            'versions': ['generic-node/v0.2'],
            'inputs': [
                {'datatype': 'BYTES', 'name': 'input-1', 'shape': [1, 20]},
                {'datatype': 'BYTES', 'name': 'input-2', 'shape': [1, 30]}
            ],
            'outputs': [{'datatype': 'BYTES', 'name': 'output', 'shape': [1]}]
        },
        'node-one': {
            'name': 'node-one',
            'platform': 'seldon-custom',
            'versions': ['generic-node/v0.2'],
            'inputs': [{'datatype': 'BYTES', 'name': 'input', 'shape': [1, 10]}],
            'outputs': [{'datatype': 'BYTES', 'name': 'output', 'shape': [1, 20]}]
        },
        'node-two': {
            'name': 'node-two',
            'platform': 'seldon-custom',
            'versions': ['generic-node/v0.2'],
            'inputs': [{'datatype': 'BYTES', 'name': 'input', 'shape': [1, 10]}],
            'outputs': [{'datatype': 'BYTES', 'name': 'output', 'shape': [1, 30]}]
        }
    },
    'graphinputs': [{'datatype': 'BYTES', 'name': 'input', 'shape': [1, 10]}],
    'graphoutputs': [{'datatype': 'BYTES', 'name': 'output', 'shape': [1]}]
}

meta

{'name': 'example',
 'models': {'node-combiner': {'name': 'node-combiner',
   'platform': 'seldon-custom',
   'versions': ['generic-node/v0.2'],
   'inputs': [{'datatype': 'BYTES', 'name': 'input-1', 'shape': [1, 20]},
    {'datatype': 'BYTES', 'name': 'input-2', 'shape': [1, 30]}],
   'outputs': [{'datatype': 'BYTES', 'name': 'output', 'shape': [1]}]},
  'node-one': {'name': 'node-one',
   'platform': 'seldon-custom',
   'versions': ['generic-node/v0.2'],
   'inputs': [{'datatype': 'BYTES', 'name': 'input', 'shape': [1, 10]}],
   'outputs': [{'datatype': 'BYTES', 'name': 'output', 'shape': [1, 20]}]},
  'node-two': {'name': 'node-two',
   'platform': 'seldon-custom',
   'versions': ['generic-node/v0.2'],
   'inputs': [{'datatype': 'BYTES', 'name': 'input', 'shape': [1, 10]}],
   'outputs': [{'datatype': 'BYTES', 'name': 'output', 'shape': [1, 30]}]}},
 'graphinputs': [{'datatype': 'BYTES', 'name': 'input', 'shape': [1, 10]}],
 'graphoutputs': [{'datatype': 'BYTES', 'name': 'output', '

## Router with two models

In this example request `x` is passed by `router` to one of its children.

Router then returns children output `y1` or `y2` as graph's output `y`.

Here we assume that all children accepts similarly structured input and retuna similarly structured output.

![router](./images/router.png)

In [15]:
%%writefile graph-metadata/router.yaml

apiVersion: machinelearning.seldon.io/v1
kind: SeldonDeployment
metadata:
  name: graph-metadata-router
spec:
  name: test-deployment
  predictors:
  - componentSpecs:
    - spec:
        containers:
        - image: seldonio/metadata-generic-node:0.2
          name: node-router
        - image: seldonio/metadata-generic-node:0.2
          name: node-one
          env:
          - name: MODEL_METADATA
            value: |
              {"name": "node-one",
               "versions": ["generic-node/v0.2"],
               "platform": "seldon-custom",
               "inputs": [{"name": "input", "datatype": "BYTES", "shape": [1, 10]}],
               "outputs": [{"name": "output", "datatype": "BYTES", "shape": [1, 20]}]}            
        - image: seldonio/metadata-generic-node:0.2
          name: node-two
          env:
          - name: MODEL_METADATA
            value: |
              {"name": "node-two",
               "versions": ["generic-node/v0.2"],
               "platform": "seldon-custom",
               "inputs": [{"name": "input", "datatype": "BYTES", "shape": [1, 10]}],
               "outputs": [{"name": "output", "datatype": "BYTES", "shape": [1, 20]}]}               
    graph:
      name: node-router
      type: ROUTER
      children:
      - name: node-one
        type: MODEL
        children: []   
      - name: node-two
        type: MODEL
        children: []   
    name: example
    replicas: 1

Overwriting graph-metadata/router.yaml


In [16]:
%%bash

kubectl apply -f graph-metadata/router.yaml

dep_name=$(kubectl get deploy -l seldon-deployment-id=graph-metadata-router -o jsonpath='{.items[0].metadata.name}')
kubectl rollout status deploy/${dep_name}

seldondeployment.machinelearning.seldon.io/graph-metadata-router configured
deployment "graph-metadata-router-example-0-node-router-node-one-node-two" successfully rolled out


In [17]:
import requests
meta = requests.get("http://localhost:8003/seldon/seldon/graph-metadata-router/api/v1.0/metadata")
meta = meta.json()

assert meta == {
    'name': 'example',
    'models': {
        'node-router': {
            'name': 'seldonio/metadata-generic-node',
            'versions': ['0.2'],
            'inputs': [],
            'outputs': [],
        },
        'node-one': {
            'name': 'node-one',
            'platform': 'seldon-custom',
            'versions': ['generic-node/v0.2'],
            'inputs': [{'datatype': 'BYTES', 'name': 'input', 'shape': [1, 10]}],
            'outputs': [{'datatype': 'BYTES', 'name': 'output', 'shape': [1, 20]}]
        },
        'node-two': {
            'name': 'node-two',
            'platform': 'seldon-custom',
            'versions': ['generic-node/v0.2'],
            'inputs': [{'datatype': 'BYTES', 'name': 'input', 'shape': [1, 10]}],
            'outputs': [{'datatype': 'BYTES', 'name': 'output', 'shape': [1, 20]}]
        }
    },
    'graphinputs': [{'datatype': 'BYTES', 'name': 'input', 'shape': [1, 10]}],
    'graphoutputs': [{'datatype': 'BYTES', 'name': 'output', 'shape': [1, 20]}]
}

meta

{'name': 'example',
 'models': {'node-one': {'name': 'node-one',
   'platform': 'seldon-custom',
   'versions': ['generic-node/v0.2'],
   'inputs': [{'datatype': 'BYTES', 'name': 'input', 'shape': [1, 10]}],
   'outputs': [{'datatype': 'BYTES', 'name': 'output', 'shape': [1, 20]}]},
  'node-router': {'name': 'seldonio/metadata-generic-node',
   'versions': ['0.2'],
   'inputs': [],
   'outputs': []},
  'node-two': {'name': 'node-two',
   'platform': 'seldon-custom',
   'versions': ['generic-node/v0.2'],
   'inputs': [{'datatype': 'BYTES', 'name': 'input', 'shape': [1, 10]}],
   'outputs': [{'datatype': 'BYTES', 'name': 'output', 'shape': [1, 20]}]}},
 'graphinputs': [{'datatype': 'BYTES', 'name': 'input', 'shape': [1, 10]}],
 'graphoutputs': [{'datatype': 'BYTES', 'name': 'output', 'shape': [1, 20]}]}

## Input Transformer

Input transformers work almost exactly the same as chained nodes, see two-level example above.

Following graph is presented in a way that is suppose to make next example (output transfomer) more intuitive.

![input-transformer](./images/input-transformer.png)

In [18]:
%%writefile graph-metadata/input-transformer.yaml

apiVersion: machinelearning.seldon.io/v1
kind: SeldonDeployment
metadata:
  name: graph-metadata-input
spec:
  name: test-deployment
  predictors:
  - componentSpecs:
    - spec:
        containers:
        - image: seldonio/metadata-generic-node:0.2
          name: node-input-transformer
          env:
          - name: MODEL_METADATA
            value: |
              {"name": "node-input-transformer",
               "versions": ["generic-node/v0.2"],
               "platform": "seldon-custom",
               "inputs": [{"name": "input", "datatype": "BYTES", "shape": [1, 10]}],
               "outputs": [{"name": "output", "datatype": "BYTES", "shape": [1, 20]}]}            
        - image: seldonio/metadata-generic-node:0.2
          name: node
          env:
          - name: MODEL_METADATA
            value: |
              {"name": "node",
               "versions": ["generic-node/v0.2"],
               "platform": "seldon-custom",
               "inputs": [{"name": "input", "datatype": "BYTES", "shape": [1, 20]}],
               "outputs": [{"name": "output", "datatype": "BYTES", "shape": [1]}]}               
    graph:
      name: node-input-transformer
      type: TRANSFORMER
      children:
      - name: node
        type: MODEL
        children: []   
    name: example
    replicas: 1

Overwriting graph-metadata/input-transformer.yaml


In [19]:
%%bash

kubectl apply -f graph-metadata/input-transformer.yaml

dep_name=$(kubectl get deploy -l seldon-deployment-id=graph-metadata-input -o jsonpath='{.items[0].metadata.name}')
kubectl rollout status deploy/${dep_name}

seldondeployment.machinelearning.seldon.io/graph-metadata-input configured
deployment "graph-metadata-input-example-0-node-input-transformer-node" successfully rolled out


In [20]:
import requests
meta = requests.get("http://localhost:8003/seldon/seldon/graph-metadata-input/api/v1.0/metadata")
meta = meta.json()

assert meta == {
    'name': 'example',
    'models': {
        'node-input-transformer': {
            'name': 'node-input-transformer',
            'platform': 'seldon-custom',
            'versions': ['generic-node/v0.2'],
            'inputs': [{'datatype': 'BYTES', 'name': 'input', 'shape': [1, 10]}],
            'outputs': [{'datatype': 'BYTES', 'name': 'output', 'shape': [1, 20]}]
        },
        'node': {
            'name': 'node',
            'platform': 'seldon-custom',
            'versions': ['generic-node/v0.2'],
            'inputs': [{'datatype': 'BYTES', 'name': 'input', 'shape': [1, 20]}],
            'outputs': [{'datatype': 'BYTES', 'name': 'output', 'shape': [1]}]
        }
    },
    'graphinputs': [{'datatype': 'BYTES', 'name': 'input', 'shape': [1, 10]}],
    'graphoutputs': [{'datatype': 'BYTES', 'name': 'output', 'shape': [1]}]}

meta

{'name': 'example',
 'models': {'node': {'name': 'node',
   'platform': 'seldon-custom',
   'versions': ['generic-node/v0.2'],
   'inputs': [{'datatype': 'BYTES', 'name': 'input', 'shape': [1, 20]}],
   'outputs': [{'datatype': 'BYTES', 'name': 'output', 'shape': [1]}]},
  'node-input-transformer': {'name': 'node-input-transformer',
   'platform': 'seldon-custom',
   'versions': ['generic-node/v0.2'],
   'inputs': [{'datatype': 'BYTES', 'name': 'input', 'shape': [1, 10]}],
   'outputs': [{'datatype': 'BYTES', 'name': 'output', 'shape': [1, 20]}]}},
 'graphinputs': [{'datatype': 'BYTES', 'name': 'input', 'shape': [1, 10]}],
 'graphoutputs': [{'datatype': 'BYTES', 'name': 'output', 'shape': [1]}]}

## Output Transformer

Output transformers work almost exactly opposite as chained nodes in the two-level example above.

Input `x` is first passed to the model that is child of the `output-transformer` before it is passed to it.

![output-transformer](./images/output-transformer.png)

In [21]:
%%writefile graph-metadata/output-transformer.yaml

apiVersion: machinelearning.seldon.io/v1
kind: SeldonDeployment
metadata:
  name: graph-metadata-output
spec:
  name: test-deployment
  predictors:
  - componentSpecs:
    - spec:
        containers:
        - image: seldonio/metadata-generic-node:0.2
          name: node-output-transformer
          env:
          - name: MODEL_METADATA
            value: |
              {"name": "node-output-transformer",
               "versions": ["generic-node/v0.2"],
               "platform": "seldon-custom",
               "inputs": [{"name": "input", "datatype": "BYTES", "shape": [1, 20]}],
               "outputs": [{"name": "output", "datatype": "BYTES", "shape": [1]}]}               
        - image: seldonio/metadata-generic-node:0.2
          name: node
          env:
          - name: MODEL_METADATA
            value: |
              {"name": "node",
               "versions": ["generic-node/v0.2"],
               "platform": "seldon-custom",
               "inputs": [{"name": "input", "datatype": "BYTES", "shape": [1, 10]}],
               "outputs": [{"name": "output", "datatype": "BYTES", "shape": [1, 20]}]}            
    graph:
      name: node-output-transformer
      type: OUTPUT_TRANSFORMER
      children:
      - name: node
        type: MODEL
        children: []   
    name: example
    replicas: 1

Overwriting graph-metadata/output-transformer.yaml


In [22]:
%%bash

kubectl apply -f graph-metadata/output-transformer.yaml

dep_name=$(kubectl get deploy -l seldon-deployment-id=graph-metadata-output -o jsonpath='{.items[0].metadata.name}')
kubectl rollout status deploy/${dep_name}

seldondeployment.machinelearning.seldon.io/graph-metadata-output configured
deployment "graph-metadata-output-example-0-node-output-transformer-node" successfully rolled out


In [25]:
import requests
meta = requests.get("http://localhost:8003/seldon/seldon/graph-metadata-output/api/v1.0/metadata")
meta = meta.json()

assert meta == {
    'name': 'example',
    'models': {
        'node-output-transformer': {
            'name': 'node-output-transformer',
            'platform': 'seldon-custom',
            'versions': ['generic-node/v0.2'],
            'inputs': [{'datatype': 'BYTES', 'name': 'input', 'shape': [1, 20]}],
            'outputs': [{'datatype': 'BYTES', 'name': 'output', 'shape': [1]}]
        },
        'node': {
            'name': 'node',
            'platform': 'seldon-custom',
            'versions': ['generic-node/v0.2'],
            'inputs': [{'datatype': 'BYTES', 'name': 'input', 'shape': [1, 10]}],
            'outputs': [{'datatype': 'BYTES', 'name': 'output', 'shape': [1, 20]}]
        }
    },
    'graphinputs': [{'datatype': 'BYTES', 'name': 'input', 'shape': [1, 10]}],
    'graphoutputs': [{'datatype': 'BYTES', 'name': 'output', 'shape': [1]}]}

meta

{'name': 'example',
 'models': {'node': {'name': 'node',
   'platform': 'seldon-custom',
   'versions': ['generic-node/v0.2'],
   'inputs': [{'datatype': 'BYTES', 'name': 'input', 'shape': [1, 10]}],
   'outputs': [{'datatype': 'BYTES', 'name': 'output', 'shape': [1, 20]}]},
  'node-output-transformer': {'name': 'node-output-transformer',
   'platform': 'seldon-custom',
   'versions': ['generic-node/v0.2'],
   'inputs': [{'datatype': 'BYTES', 'name': 'input', 'shape': [1, 20]}],
   'outputs': [{'datatype': 'BYTES', 'name': 'output', 'shape': [1]}]}},
 'graphinputs': [{'datatype': 'BYTES', 'name': 'input', 'shape': [1, 10]}],
 'graphoutputs': [{'datatype': 'BYTES', 'name': 'output', 'shape': [1]}]}

## Cleanup resources

In [26]:
%%bash
kubectl delete -f graph-metadata/

seldondeployment.machinelearning.seldon.io "graph-metadata-combiner" deleted
seldondeployment.machinelearning.seldon.io "graph-metadata-input" deleted
seldondeployment.machinelearning.seldon.io "graph-metadata-output" deleted
seldondeployment.machinelearning.seldon.io "graph-metadata-router" deleted
seldondeployment.machinelearning.seldon.io "graph-metadata-single" deleted
seldondeployment.machinelearning.seldon.io "graph-metadata-two-levels" deleted
